<a href="https://colab.research.google.com/github/adityasuru95/SurgTools/blob/main/xml_parse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>